# **Name: Ujjwal Kishor Sahoo | Roll Number: 21293 | NLP-Assignment 2**

In [52]:
!pip install keras-tuner --upgrade

## **Loading the dataset**

In [53]:
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/val.csv

--2025-04-04 10:28:34--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406615 (397K) [text/plain]
Saving to: ‘train.csv.3’

train.csv.3         100%[===================>] 397.08K  --.-KB/s    in 0.05s   

2025-04-04 10:28:34 (8.58 MB/s) - ‘train.csv.3’ saved [406615/406615]

--2025-04-04 10:28:34--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49821 (

In [54]:
pip install keras

In [55]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score

# Machine learning and neural networks
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Embedding, Dense, Flatten, Input, Concatenate, Dropout)
from tensorflow.keras.optimizers import Adam, AdamW, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow.keras.backend as K
from keras.metrics import Precision, Recall
from kerastuner import HyperParameters
from kerastuner.tuners import RandomSearch
from keras_tuner import Objective
from sklearn.utils import class_weight



# Text processing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Hyperparameter tuning
import keras_tuner as kt
from keras_tuner import RandomSearch, Objective

In [56]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Hate Dataset

In [57]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv')
hate_test_data = pd.read_csv('/content/val.csv')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing**

In [58]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [59]:
# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)

In [60]:
# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_hate.npy', trained_embeddings)

Epoch 1/5
4395/4395 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - loss: 0.2095
Epoch 2/5


In [ ]:
# Load the custom-trained Word2Vec embeddings
trained_embeddings = np.load('word2vec_embeddings_hate.npy')

In [ ]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)


In [ ]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Defining custom evalutation metric**

In [ ]:
# Define a custom macro F1 score function
def macro_f1_score(y_true, y_pred):
    # Ensure both y_true and y_pred are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Convert predictions to binary (0 or 1)
    y_pred_bin = tf.round(y_pred)

    # Calculate precision, recall, and F1 score for each class
    def f1(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())

        f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_val

    f1_per_class = f1(y_true, y_pred_bin)
    return K.mean(f1_per_class)  # Macro F1 score (mean across all classes)

## **Training**

In [ ]:
# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_hate',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()

## **Saving the model**

In [ ]:
# Save FFNN Model
best_model_ffnn.save('best_model_ffnn.keras')

## **Loading the saved models**

In [ ]:
# Load the model from the saved files
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn.keras', custom_objects={'macro_f1_score': macro_f1_score})

In [ ]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

# Humor Dataset

In [ ]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv.1')
hate_test_data = pd.read_csv('/content/val.csv.1')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing**

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [ ]:
# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)

In [ ]:
# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

In [ ]:
# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_humor.npy', trained_embeddings)

In [ ]:
# Load the custom-trained Word2Vec embeddings
trained_embeddings = np.load('word2vec_embeddings_humor.npy')

In [ ]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)


In [ ]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Defining custom evalutation metric**

In [ ]:
# Define a custom macro F1 score function
def macro_f1_score(y_true, y_pred):
    # Ensure both y_true and y_pred are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Convert predictions to binary (0 or 1)
    y_pred_bin = tf.round(y_pred)

    # Calculate precision, recall, and F1 score for each class
    def f1(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())

        f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_val

    f1_per_class = f1(y_true, y_pred_bin)
    return K.mean(f1_per_class)  # Macro F1 score (mean across all classes)

## **Training**

In [ ]:
# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_humor',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()

## **Saving the model**

In [ ]:
# Save FFNN Model
best_model_ffnn.save('best_model_ffnn.keras')

## **Loading the saved models**

In [ ]:
# Load the model from the saved files
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn.keras', custom_objects={'macro_f1_score': macro_f1_score})

In [ ]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

# Sarcasm Dataset

In [ ]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv.2')
hate_test_data = pd.read_csv('/content/val.csv.2')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing**

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [ ]:
# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)

In [ ]:
# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

In [ ]:
# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_sarcasm.npy', trained_embeddings)

In [ ]:
# Load the custom-trained Word2Vec embeddings
trained_embeddings = np.load('word2vec_embeddings_sarcasm.npy')

In [ ]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)

In [ ]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Defining custom evalutation metric**

In [ ]:
# Define a custom macro F1 score function
def macro_f1_score(y_true, y_pred):
    # Ensure both y_true and y_pred are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Convert predictions to binary (0 or 1)
    y_pred_bin = tf.round(y_pred)

    # Calculate precision, recall, and F1 score for each class
    def f1(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())

        f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_val

    f1_per_class = f1(y_true, y_pred_bin)
    return K.mean(f1_per_class)  # Macro F1 score (mean across all classes)

## **Training**

In [ ]:
# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_sarcasm',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()

## **Saving the model**

In [ ]:
# Save FFNN Model
best_model_ffnn.save('best_model_ffnn.keras')

## **Loading the saved models**

In [ ]:
# Load the model from the saved files
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn.keras', custom_objects={'macro_f1_score': macro_f1_score})

In [ ]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")